In [1]:
from arcgis.gis import GIS
import arcgis.network as network
from arcgis.geocoding import geocode, reverse_geocode
from arcgis.features import Feature, FeatureSet

import sys
sys.path.append('../../')
from utils import get_config

from IPython.display import HTML


In [26]:
def nearest_facilities(originating_address, all_facility_addresses, default_target_facility_count=4):
    username = get_config("arcgis","username")
    password = get_config("arcgis","passkey")
    gis = GIS("https://ucsdonline.maps.arcgis.com/home", username=username, password=password)
    
    analysis_url = gis.properties.helperServices.closestFacility.url
    cf_layer = network.ClosestFacilityLayer(analysis_url, gis=gis)

    facility_feat_list = []
    matched_facility_address = []

    for address in all_facility_addresses:
        hit = geocode(address, max_locations=1)[0]
        facility_feat = Feature(geometry=hit['location'], attributes=hit['attributes'])
        matched_facility_address.append(facility_feat.attributes['Match_addr'])
        facility_feat_list.append(facility_feat)

    facility_fset = FeatureSet(features=facility_feat_list, 
                                geometry_type='esriGeometryPoint', 
                                spatial_reference={'latestWkid': 4326})
        
    # originating_address = '581 Moss St, 91911, Chula Vista, CA'
    originating_matched_address = geocode(originating_address, max_locations=1)[0]
        
    originating_address_feature = Feature(geometry=originating_matched_address['location'], attributes=originating_matched_address['attributes'])
    originating_address_fset = FeatureSet([originating_address_feature], geometry_type='esriGeometryPoint',
                          spatial_reference={'latestWkid': 4326})
    
    result = cf_layer.solve_closest_facility(incidents=originating_address_fset,
                                        facilities=facility_fset,
                                        default_target_facility_count=len(all_facility_addresses),
                                        return_facilities=True,
                                        impedance_attribute_name='TravelTime',
                                        accumulate_attribute_names=['Miles','TravelTime'])
    line_feat_list = []
    for line_dict in result['routes']['features']:
        f1 = Feature(line_dict['geometry'], line_dict['attributes'])
        line_feat_list.append(f1)
        
    routes_fset = FeatureSet(line_feat_list, 
                         geometry_type=result['routes']['geometryType'],
                         spatial_reference= result['routes']['spatialReference'])
    df1 = routes_fset.sdf
    
    df1['facility_address'] = matched_facility_address
    df1['originating_address'] = [originating_address_feature.attributes['Match_addr'] for i in range(len(all_facility_addresses))]
    df1 = df1[['facility_address','originating_address','Total_Miles','Total_TravelTime']]
    return df1


In [41]:
def nearest_facility(originating_address, all_facility_addresses):
    username = get_config("arcgis","username")
    password = get_config("arcgis","passkey")
    gis = GIS("https://ucsdonline.maps.arcgis.com/home", username=username, password=password)
    
    analysis_url = gis.properties.helperServices.closestFacility.url
    cf_layer = network.ClosestFacilityLayer(analysis_url, gis=gis)

    facility_feat_list = []
    matched_facility_address = []

    for address in all_facility_addresses:
        hit = geocode(address, max_locations=1)[0]
        facility_feat = Feature(geometry=hit['location'], attributes=hit['attributes'])
        matched_facility_address.append(facility_feat.attributes['Match_addr'])
        facility_feat_list.append(facility_feat)

    facility_fset = FeatureSet(features=facility_feat_list, 
                                geometry_type='esriGeometryPoint', 
                                spatial_reference={'latestWkid': 4326})
        
    # originating_address = '581 Moss St, 91911, Chula Vista, CA'
    originating_matched_address = geocode(originating_address, max_locations=1)[0]
        
    originating_address_feature = Feature(geometry=originating_matched_address['location'], attributes=originating_matched_address['attributes'])
    originating_address_fset = FeatureSet([originating_address_feature], geometry_type='esriGeometryPoint',
                          spatial_reference={'latestWkid': 4326})
    
    result = cf_layer.solve_closest_facility(incidents=originating_address_fset,
                                        facilities=facility_fset,
                                        default_target_facility_count=len(all_facility_addresses),
                                        return_facilities=True,
                                        impedance_attribute_name='TravelTime',
                                        accumulate_attribute_names=['Miles','TravelTime'])
    line_feat_list = []
    for line_dict in result['routes']['features']:
        f1 = Feature(line_dict['geometry'], line_dict['attributes'])
        line_feat_list.append(f1)
        
    routes_fset = FeatureSet(line_feat_list, 
                         geometry_type=result['routes']['geometryType'],
                         spatial_reference= result['routes']['spatialReference'])
    df1 = routes_fset.sdf
    
    df1['facility_address'] = matched_facility_address
    df1['originating_address'] = [originating_address_feature.attributes['Match_addr'] for i in range(len(all_facility_addresses))]
    df1 = df1[['facility_address','originating_address','Total_Miles','Total_TravelTime']]
    
    # select the row with the minimum Total_Miles
    min_miles_index = df1['Total_Miles'].idxmin()
    min_miles_row = df1.loc[min_miles_index]

    return min_miles_row.to_json()


In [32]:
sba_addresses = ['550 West C St. suite 550, San Diego',
                 '880 National City Blvd, San Diego',
                 '530 B St. Suite 700, San Diego',
                 '7675 Dagget St. Suite 340, San Diego',
                 '1178 Caminito Alvarez, San Diego']

In [33]:
originating_address = '581 Moss St, 91911, Chula Vista, CA'

In [42]:
print(nearest_facility(originating_address,sba_addresses))

{"facility_address":"550 W C St, Ste 550, San Diego, California, 92101","originating_address":"581 Moss St, Chula Vista, California, 91911","Total_Miles":5.3147842882,"Total_TravelTime":8.2612650342}
